<a href="https://colab.research.google.com/github/cawcawlabs/ouster_lidar_appnotes/blob/main/google_colab_demos/Centerpoint_3D_obj_detection_OS1_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/cawcawlabs/CenterPoint.git
!nvcc -V #check cuda version, done for 11.1
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install torchtext==0.9.1
#https://github.com/qubvel/segmentation_models.pytorch/issues/424
import os
%cd /content/CenterPoint/
!pip install -r requirements.txt

%cd /content/
os.environ['PYTHONPATH'] += ":/content/CenterPoint/"
!echo $PYTHONPATH

!git clone https://github.com/cawcawlabs/nuscenes-devkit.git
os.environ['PYTHONPATH'] += ":/content/nuscenes-devkit/python-sdk"
!echo $PYTHONPATH

%cd /content/CenterPoint/
!export PATH=/usr/local/cuda-11.0/bin:$PATH
!export CUDA_PATH=/usr/local/cuda-11.0
!export CUDA_HOME=/usr/local/cuda-11.0
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH
!bash setup.sh
%cd /content/
!git clone https://github.com/NVIDIA/apex
%cd /content/apex
!git checkout 5633f6
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
#getting the latest spconv-cu111
!pip install spconv-cu111
!apt-get install libboost-all-dev

%cd /content/CenterPoint
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ctp/demo.zip
!unzip /content/CenterPoint/demo.zip
%cd /content/CenterPoint/
%mkdir work_dirs
%cd /content/CenterPoint/work_dirs/
%mkdir centerpoint_pillar_512_demo
%cd /content/CenterPoint/work_dirs/centerpoint_pillar_512_demo
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ctp/centerpoint_pillar_512_demo.zip
!unzip ./centerpoint_pillar_512_demo.zip
%cd /content/CenterPoint/work_dirs/
%mkdir ouster_data


#TEST INFERENCE #1 (Ouster REV6 data, calibrated reflectivity) <br> 
generate nuscenes pcd.bin from ouster pcap recording then run inference.<br>
Calibrated Reflectivity, results stored as png per frame on /CenterPoint/demo/filexxxxx.png, and /CenterPoint/video.avi

In [ ]:
#%cd /content/
#!wget https://data.ouster.io/sdk-samples/Rev-05/OS1-128_Rev-05_Urban-Drive/OS1-128_Rev-05_Urban-Drive.zip
#!unzip /content/OS1-128_Rev-05_Urban-Drive.zip

In [ ]:
#REV6 urban drive sample, calibrated reflectivity
%cd /content/
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ctp/OS-1-128_v2.3.0_1024x10_20220419_160551-000.pcap
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ctp/OS-1-128_v2.3.0_1024x10_20220419_160551.json

In [ ]:
!pip install 'ouster-sdk'

In [ ]:
#PREP pcaps to nuscenes compatible pcd.bin

import numpy as np
from ouster import client
from ouster import pcap
from contextlib import closing
from more_itertools import nth

#pcap_path = '/content/OS1-128_Rev-05_Urban-Drive.pcap'
#metadata_path = '/content/OS1-128_Rev-05_Urban-Drive.json'

pcap_path = '/content/OS-1-128_v2.3.0_1024x10_20220419_160551-000.pcap'
metadata_path = '/content/OS-1-128_v2.3.0_1024x10_20220419_160551.json'



with open(metadata_path, 'r') as f:
    info = client.SensorInfo(f.read())

source = pcap.Pcap(pcap_path, info)

with closing(client.Scans(source)) as scans:
    index = 0
    for scan in scans:
      print(index)
      #print(scan.frame_id)
      xyzlut = client.XYZLut(scans.metadata)
      #destagger
      signal  = client.destagger(scans.metadata,scan.field(client.ChanField.REFLECTIVITY))
      #signal = (signal / np.max(signal)).astype(np.float32) #will probably determine accuracy here
      xyz_destaggered = client.destagger(scans.metadata, xyzlut(scan))
      [x, y, z] = [c.flatten() for c in np.dsplit(xyz_destaggered, 3)]
      signal = signal.flatten()
      point_cloud = np.asarray([x,y,z,signal], dtype =  np.float32).transpose() #find ways to add zero

      #print(point_cloud.shape) #262144X4

      zero_pad = np.zeros((len(point_cloud),1),dtype=np.float32) #fake ring data (not used)

      nuscenes = np.hstack((point_cloud,zero_pad))

      file_index = str(index).zfill(5) #00000 scenes
      nuscenes.tofile('/content/CenterPoint/work_dirs/ouster_data/ouster_{}.pcd.bin'.format(file_index))

      index+=1


In [ ]:
%cd /content/CenterPoint
!python ./ouster_single_inference.py

/content/CenterPoint
/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
No Tensorflow
Deformable Convolution not built!
Use HM Bias:  -2.19
  0% 0/720 [00:00<?, ?it/s]/content/CenterPoint/work_dirs/ouster_data/ouster_00000.pcd.bin
{'boxes': array([[-7.39154434e+00,  5.05042267e+00, -1.26426589e+00, ...,
         9.72824576e-09,  2.99704062e-09,  1.57614589e+00],
       [ 2.26502609e+00,  5.14960861e+00, -8.91525984e-01, ...,
        -2.40401077e+00, -1.07591033e-01,  1.55452466e+00],
       [-1.08146133e+01, -3.40338898e+00, -1.30688608e+00, ...,
         9.72824576e-09,  2.99704062e-09, -1.55909109e+00],
       ...,
       [ 2.18251915e+01,  1.47205925e+01, -8.94743800e-01, ...,
        -2.82468856e-04, -3.85179010e-04,  1.19341016e+00],
       [-1.11037445e+00, -1.00735664e+01, -6.66422367e-01, ...,
         1.20257447e-02,  1.86759867e-02, -1.50